In [1]:
%run InitialSetup.ipynb

In [179]:
# Construct Surface between 5X5 and 20x20 MaturityXTenor tuples
it_range = [i for i in range(5, 11)] + [15, 20]

# Create tuples of dates
iter_list = list(itertools.product(*[it_range, it_range]))

# Create similar names to the titles of the vol_data file
vol_filt = [str(i)+'Y '+str(j)+'Y' for i, j in iter_list]

# Filter vol_data based on criteria
vol_surface = vol_data[vol_filt]

# Select Training Data up to 2017
vol_surface = vol_surface[:'2016-12-31']

#Select Test Data
vol_surface_test = vol_data[vol_filt]['2016-12-31':]

In [48]:
# Vectorize functions
Hagan_Vect = np.vectorize(sigma_CEV_Hagan)
Black_Vect = np.vectorize(payer_Black)
Chi_Square_Vect = np.vectorize(payer_chi_square)

In [ ]:
# Set data as an array 
row_wise_vol_data = np.array(vol_surface.values.flatten())/10000

# Define list of maturities and tenors
eval_dates = [DateParser.parseFormatted(datetime.strftime(date,'%Y-%m-%d'), '%Y-%m-%d') for date in vol_surface.index]
eval_dates = np.array(eval_dates)
full_eval_dates = np.repeat(eval_dates, vol_surface.shape[1])

#Repeat each date for each swaption we selected in vol_surface
maturity_array = np.array([item[1] for item in iter_list]*len(vol_surface))
maturities = [date + Period(int(i), Years) for date, i in zip(full_eval_dates, maturity_array)]
maturities = np.array(maturities)
#tenor_array = np.array([item[0] for item in iter_list]*len(vol_data))

#Forward Rates to be added

#Construct the full calibration data
full_calib_data = np.matrix(np.stack([full_eval_dates,maturities, row_wise_vol_data], axis = 1))

In [247]:
# Find optimal sigma in Hagan Approx 
S0 = 0.025
K = 0.025
sigma = 0.09999978695526811
sigma2 = 0.07
beta = 0.35

# Test0 = sigma and Test1 = beta
# Construct the function to be optimized
opt_func = lambda test: np.sum(np.square(Hagan_Vect(S0, K ,test[0], test[1], 
                                                full_calib_data[:,0], 
                                                full_calib_data[:,1]) 
                                                - full_calib_data[:,2]))

In [ ]:
'''3) Least Squares Optimization'''
start_LSO = time.time()
value_LS = optimize.least_squares(opt_func, x0 = [0.01,0.5],
                       ftol = 1e-9, 
                       xtol=1e-09, 
                       gtol=1e-09, 
                       method = 'trf')
end_LSO = time.time()

'''4) Dual Annealing Optimization'''
start_DAO = time.time()
value_DAO = optimize.dual_annealing(opt_func_100, bounds = [(0, 1),(0, 1)]) 
#                        ftol=1e-09, 
#                        ftol=1e-09)
end_DAO = time.time()

'''5) Fmin tnc Optimization'''
start_TNC = time.time()
value_TNC = optimize.fmin_tnc(opt_func_100, 
                  x0 = [0.01,0.5],
                  bounds = [(0, 1),(0, 1)],
                  approx_grad = True,
                  epsilon= 1e-9)
end_TNC = time.time()

'''6) Fmin l_bfgs_b Optimization'''
start_BFGS = time.time()
value_BFGS = optimize.fmin_l_bfgs_b(opt_func_100, 
                  x0 = [0.01,0.5],
                  bounds = [(0, 1),(0, 1)],
                  approx_grad = True)
end_BFGS = time.time()

'''7) Powell Optimization'''
start_Pow = time.time()
value_Pow = optimize.fmin_powell(opt_func_100, 
                  x0 = [0.01,0.5],
                  maxiter = 1e+9, 
                  xtol=1e-09, 
                  ftol=1e-09)
end_Pow = time.time()

'''8) Differential Evolution Optimization'''
start_DEO = time.time()
value_DEO = optimize.fmin_powell(opt_func_100, 
                     x0 = [0.01,0.5],
                  maxiter = 1e+9)
end_DEO = time.time()

'''10) SLSQP Optimization'''
start_SLSQP = time.time()
value_SLSQP = optimize.fmin_slsqp(opt_func_100, 
                  x0 = [0.01,0.5],
                  bounds = [(0, 1),(0, 1)],
                  iter = 1e+9)
end_SLSQP = time.time()

In [249]:
from prettytable import PrettyTable
names = ['Least Squares', 'Dual Annealing', 'TNC optimization', 'BFGS', 'Powell', 'Differential Evolution',
        'SLSQP']
error = [opt_func(value_LS.x), opt_func(value_DAO.x),opt_func(value_TNC[0]), opt_func(value_BFGS[0]),
         opt_func(value_Pow),opt_func(value_DEO),opt_func(value_SLSQP)]

sigma = [list(value_LS.x)[0],list(value_DAO.x)[0], list(value_TNC[0])[0], list(value_BFGS[0])[0],
              list(value_Pow)[0], list(value_DEO)[0], list(value_SLSQP)[0] ]
beta = [list(value_LS.x)[1],list(value_DAO.x)[1], list(value_TNC[0])[1], list(value_BFGS[0])[1],
              list(value_Pow)[1], list(value_DEO)[1], list(value_SLSQP)[1]]

headers = ["Algorithms", "Sigma", "Beta", "Error"]
t = PrettyTable()
t.add_column(headers[0],  list(names))
t.add_column(headers[1],  list(sigma))
t.add_column(headers[2],  list(beta))
t.add_column(headers[3],  list(error))
print(t)

+------------------------+------------------------+--------------------+---------------------+
|       Algorithms       |         Sigma          |        Beta        |        Error        |
+------------------------+------------------------+--------------------+---------------------+
|     Least Squares      | 0.0005395398569453848  | 0.5002988223601984 | 0.33668462847629843 |
|     Dual Annealing     |  0.001838792145459032  | 0.9999407037021425 |  0.5807118829688934 |
|    TNC optimization    | 0.00029118397308863786 | 0.5003552075026217 |  0.580710681404637  |
|          BFGS          |  0.00181733159968711   | 0.9967586846992861 |  0.5807127738442021 |
|         Powell         | 0.0002908029537403325  | 0.5000000024570254 |  0.5807101468415898 |
| Differential Evolution | 0.00029385085128537944 | 0.5028264495548495 |  0.5807101464652882 |
|         SLSQP          |  0.001213260602629589  | 0.8955181904674421 |  0.5982499740159231 |
+------------------------+------------------------

In [250]:
'Test Calibration'
S0 = 0.025
K = 0.025

# Set data as an array 
row_wise_vol_data_test = np.array(vol_surface_test.values.flatten())/10000

# Define list of maturities and tenors
eval_dates_test = [DateParser.parseFormatted(datetime.strftime(date,'%Y-%m-%d'), '%Y-%m-%d') 
              for date in vol_surface_test.index]
eval_dates_test = np.array(eval_dates_test)
full_eval_dates_test = np.repeat(eval_dates_test, vol_surface_test.shape[1])

#Repeat each date for each swaption we selected in vol_surface
maturity_array_test = np.array([item[1] for item in iter_list]*len(vol_surface_test))
maturities_test = [date + Period(int(i), Years) for date, i in zip(full_eval_dates_test, maturity_array_test)]
maturities_test = np.array(maturities_test)
#tenor_array = np.array([item[0] for item in iter_list]*len(vol_data))

#Forward Rates to be added

#Construct the full calibration data
full_calib_data_test = np.matrix(np.stack([full_eval_dates_test,
                                           maturities_test, 
                                           row_wise_vol_data_test], axis = 1))


In [252]:
"Define a new error function"
opt_func_test = lambda test: np.sum(np.square(Hagan_Vect(S0, K ,test[0], test[1], 
                                                full_calib_data_test[:,0], 
                                                full_calib_data_test[:,1]) 
                                                - full_calib_data_test[:,2]))

In [253]:
"Display the test results"
names = ['Least Squares', 'Dual Annealing', 'TNC optimization', 'BFGS', 'Powell', 'Differential Evolution',
        'SLSQP']
error = [opt_func_test(value_LS.x), opt_func_test(value_DAO.x),opt_func_test(value_TNC[0]), 
         opt_func_test(value_BFGS[0]),
         opt_func_test(value_Pow),opt_func_test(value_DEO),opt_func_test(value_SLSQP)]

headers = ["Algorithms", "Squared Error"]
t2 = PrettyTable()
t2.add_column(headers[0],  list(names))
t2.add_column(headers[1],  list(sigma))
print(t, t2)

+------------------------+------------------------+--------------------+---------------------+
|       Algorithms       |         Sigma          |        Beta        |        Error        |
+------------------------+------------------------+--------------------+---------------------+
|     Least Squares      | 0.0005395398569453848  | 0.5002988223601984 | 0.33668462847629843 |
|     Dual Annealing     |  0.001838792145459032  | 0.9999407037021425 |  0.5807118829688934 |
|    TNC optimization    | 0.00029118397308863786 | 0.5003552075026217 |  0.580710681404637  |
|          BFGS          |  0.00181733159968711   | 0.9967586846992861 |  0.5807127738442021 |
|         Powell         | 0.0002908029537403325  | 0.5000000024570254 |  0.5807101468415898 |
| Differential Evolution | 0.00029385085128537944 | 0.5028264495548495 |  0.5807101464652882 |
|         SLSQP          |  0.001213260602629589  | 0.8955181904674421 |  0.5982499740159231 |
+------------------------+------------------------

In [254]:
"TESTING THE LEVENBERG MARQUART ALGORITHM"
from lmfit import minimize, Parameters, Parameter, report_fit

#Define the Objective function
def opt_func_lm(params):
    sigma = params['sigma']
    beta = params['beta']
    
    results = (Hagan_Vect(S0, K ,params['sigma'], params['beta'], 
                                                full_calib_data[:,0], 
                                                full_calib_data[:,1]) 
                                                - full_calib_data[:,2])
    return(results)

In [ ]:
params = Parameters()
params.add('sigma',value = 0.01, min = 0, max = 1)
params.add('beta', value = 0.3, min = 0, max = 1)
out = minimize(opt_func_lm, params)